# Gallery Example: M/E₂/1 Queue (Erlang Service)

This example demonstrates an M/E₂/1 queueing system:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Erlang-2 service times (less variable than exponential)
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Erlang-2 service distribution has lower variance than exponential, representing more consistent service times.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_merl1(): Network {    """Create M/E₂/1 queueing model"""    model = Network("M/E/1")        # Block 1: nodes    source = Source(model, "mySource")    queue = Queue(model, "myQueue", SchedStrategy.FCFS)    sink = Sink(model, "mySink")        # Block 2: classes    oclass = OpenClass(model, "myClass")    # Exponential arrivals with rate λ=1    source.setArrival(oclass, Exp(1))    # Erlang-2 service with mean=0.5 and order=2    queue.setService(oclass, Erlang.fit_mean_and_order(0.5, 2))        # Block 3: topology    P = model.initRoutingMatrix()    P.addRoute(oclass, source, queue, 1.0)    P.addRoute(oclass, queue, sink, 1.0)    model.link(P)        return model, source, queue, sink, oclass# Create the modelmodel, source, queue, sink, oclass = gallery_merl1()

## Theoretical Analysis for M/E₂/1

For M/E₂/1 with:
- **Arrival rate**: λ = 1 (Exponential with mean=1)
- **Service time**: Erlang-2 with mean=0.5, order=2
- **Utilization**: ρ = λ × E[S] = 1 × 0.5 = 0.5

The Erlang-2 service distribution characteristics:
- **Mean**: 0.5
- **Variance**: (0.5)²/2 = 0.125 (lower than exponential variance = 0.25)
- **Coefficient of Variation**: C²ₛ = 1/2 = 0.5

Lower service variability typically leads to better performance than M/M/1.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Compare M/E₂/1 with M/M/1
println("\n=== Comparison with M/M/1 ===")
// Create equivalent M/M/1 model
fun create_mm1_equivalent(): Network {
    val model_mm1 = Network("M/M/1-Equivalent")
    val source = Source(model_mm1, "Source")
    val queue = Queue(model_mm1, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_mm1, "Sink")
    
    val oclass = OpenClass(model_mm1, "Class")
    source.setArrival(oclass, Exp(1))  # Same arrival rate
    queue.setService(oclass, Exp(2))   # Exponential with mean = 0.5
    
    val P = model_mm1.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_mm1.link(P)
    
    return model_mm1

val model_mm1 = create_mm1_equivalent()
val solver_merl = MVA(model)
val solver_mm1 = MVA(model_mm1)

val avg_table_merl = solver_merl.avgTable
val avg_table_mm1 = solver_mm1.avgTable

println("M/E₂/1 Results:")
println(avg_table_merl)

println("\nM/M/1 Results:")
println(avg_table_mm1)
// Extract key metrics for comparison
val merl_resp = float(avg_table_merl.iloc[1, 2])  # Queue response time
val merl_length = float(avg_table_merl.iloc[1, 3])  # Queue length

val mm1_resp = float(avg_table_mm1.iloc[1, 2])  # Queue response time
val mm1_length = float(avg_table_mm1.iloc[1, 3])  # Queue length

println(f"\nPerformance Comparison:")
println(f"Response Time: M/E₂/1={merl_resp:.4f}, M/M/1={mm1_resp:.4f}")
println(f"Queue Length: M/E₂/1={merl_length:.4f}, M/M/1={mm1_length:.4f}")
val improvement = ((mm1_resp - merl_resp) / mm1_resp * 100) if mm1_resp > merl_resp else ((merl_resp - mm1_resp) / mm1_resp * 100)
println(f"Improvement: {improvement:.1f}% {"better" if mm1_resp > merl_resp else "worse"} response time with Erlang service")

In [ ]:
// Analyze impact of Erlang order for service times
println("\n=== Impact of Service Time Variability ===")

fun create_service_erlang_model(order): Network {
    """Create M/Eₖ/1 model with specified Erlang order for service"""
    val model_erl = Network(f"M/E{order}/1")
    val source = Source(model_erl, "Source")
    val queue = Queue(model_erl, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_erl, "Sink")
    
    val oclass = OpenClass(model_erl, "Class")
    source.setArrival(oclass, Exp(1))
    queue.setService(oclass, Erlang.fit_mean_and_order(0.5, order))
    
    val P = model_erl.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_erl.link(P)
    
    return model_erl
// Test different Erlang orders for service
val orders = [1, 2, 3, 5, 10]  # Order 1 = Exponential

println("Service Order | C²ₛ  | Response Time | Queue Length")
println("-" * 50)

for order in orders:
    val model_order = create_service_erlang_model(order)
    val solver = MVA(model_order)
    val avg_table = solver.avgTable
    
    val resp_time = float(avg_table.iloc[1, 2])
    val queue_length = float(avg_table.iloc[1, 3])
    val cv_squared = 1.0 / order  # Coefficient of variation squared for Erlang
    
    println(f"     {order:2d}       | {cv_squared:.3f} |     {resp_time:.4f}    |    {queue_length:.4f}")

println("\nNote: Higher order (lower C²ₛ) leads to better performance due to reduced service variability.")
println("As order→∞, the system approaches M/D/1 (deterministic service).")

In [ ]:
// Compare with M/H₂/1 (hyperexponential service)
println("\n=== Comparison: Low vs High Service Variability ===")

fun create_mhyp1_model(): Network {
    """Create M/H₂/1 model with hyperexponential service"""
    val model_hyp = Network("M/H2/1")
    val source = Source(model_hyp, "Source")
    val queue = Queue(model_hyp, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model_hyp, "Sink")
    
    val oclass = OpenClass(model_hyp, "Class")
    source.setArrival(oclass, Exp(1))
    queue.setService(oclass, HyperExp.fit_mean_and_scv(0.5, 4))  # High variability
    
    val P = model_hyp.initRoutingMatrix()
    P.addRoute(oclass, source, queue, 1.0)
    P.addRoute(oclass, queue, sink, 1.0)
    model_hyp.link(P)
    
    return model_hyp

val model_hyp = create_mhyp1_model()
val model_erl = create_service_erlang_model(5)  # Low variability

val solver_hyp = MVA(model_hyp)
val solver_erl = MVA(model_erl)

val avg_table_hyp = solver_hyp.avgTable
val avg_table_erl = solver_erl.avgTable

val hyp_resp = float(avg_table_hyp.iloc[1, 2])
val erl_resp = float(avg_table_erl.iloc[1, 2])

println(f"Service Distribution | C²ₛ | Response Time")
println("-" * 40)
println(f"Erlang-5 (low var)   | 0.2 |    {erl_resp:.4f}")
println(f"Exponential          | 1.0 |    {mm1_resp:.4f}")
println(f"Hyperexp (high var)  | 4.0 |    {hyp_resp:.4f}")

println(f"\nVariability Impact: {(hyp_resp / erl_resp):.1f}x difference between high and low service variability")
println("This demonstrates the fundamental principle: variability degrades queueing performance.")